# LangChain Memory Patterns with Amazon Nova

This notebook demonstrates different memory patterns for maintaining conversation context with Nova models.

## Setup

Make sure you have set your environment variables:
```bash
export NOVA_API_KEY="your-api-key"
export NOVA_BASE_URL="https://api.nova.amazon.com/v1"
```

In [ ]:
%env NOVA_API_KEY="YOUR-API-KEY"
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

In [ ]:
from langchain_nova import ChatNova
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory

# Initialize the model
llm = ChatNova(
    model="nova-pro-v1",
    temperature=0.7,
    max_tokens=500,
    metadata={"demo": "memory_patterns"}
)

## 1. Manual Message History

The simplest approach: manually manage a list of messages.

In [ ]:
# Start with system message
messages = [
    SystemMessage(content="You are a helpful assistant that remembers context.")
]

# Turn 1: User introduces themselves
messages.append(HumanMessage(content="Hi! My name is Alice and I'm learning Python."))
response1 = llm.invoke(messages)
messages.append(AIMessage(content=response1.content))

print("Turn 1:")
print(f"User: Hi! My name is Alice and I'm learning Python.")
print(f"Assistant: {response1.content}\n")

In [ ]:
# Turn 2: Ask about their name (tests memory)
messages.append(HumanMessage(content="What's my name and what am I learning?"))
response2 = llm.invoke(messages)
messages.append(AIMessage(content=response2.content))

print("Turn 2:")
print(f"User: What's my name and what am I learning?")
print(f"Assistant: {response2.content}\n")

In [ ]:
# Check message history
print(f"Total messages in history: {len(messages)}")
for i, msg in enumerate(messages):
    print(f"{i+1}. {msg.type}: {msg.content[:60]}...")

## 2. Conversation Buffer Memory

LangChain's `ConversationBufferMemory` manages message history for you.

In [ ]:
# Create buffer memory
memory = ConversationBufferMemory(return_messages=True)

# Add conversations
memory.save_context(
    {"input": "Hi, I'm Bob and I live in Seattle"},
    {"output": "Hello Bob! It's nice to meet you. Seattle is a beautiful city!"}
)

memory.save_context(
    {"input": "I work as a software engineer"},
    {"output": "That's great! Software engineering is an exciting field. What technologies do you work with?"}
)

print(f"Buffer has {len(memory.chat_memory.messages)} messages")
for msg in memory.chat_memory.messages:
    print(f"{msg.type}: {msg.content}")

In [ ]:
# Use memory in a chain
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

chain = prompt | llm

response = chain.invoke({
    "history": memory.chat_memory.messages,
    "input": "Where do I live and what do I do?"
})

print(f"Assistant: {response.content}")

## 3. Conversation Summary Memory

For long conversations, `ConversationSummaryMemory` creates a rolling summary instead of keeping all messages.

In [ ]:
# Create summary memory
summary_memory = ConversationSummaryMemory(llm=llm, return_messages=True)

# Simulate a longer conversation
conversations = [
    ("What are quantum computers?", "Quantum computers use quantum bits or qubits that can be in superposition..."),
    ("How do they differ from classical computers?", "Classical computers use binary bits while quantum computers leverage quantum mechanics..."),
    ("What are practical applications?", "Applications include cryptography, drug discovery, optimization problems..."),
    ("Are they commercially available yet?", "Yes, companies like IBM and Google offer cloud access to quantum computers..."),
]

for user_msg, ai_msg in conversations:
    summary_memory.save_context({"input": user_msg}, {"output": ai_msg})

print("Conversation Summary:")
print(summary_memory.buffer)

In [ ]:
# The summary is much shorter than storing all messages
print(f"\nSummary length: {len(summary_memory.buffer)} characters")
print(f"Original conversation would be ~{sum(len(q) + len(a) for q, a in conversations)} characters")

## 4. Context Window Management

Practical pattern for managing conversation within token limits.

In [ ]:
# Set maximum messages to keep (system message + N exchanges)
MAX_MESSAGES = 10

messages = [
    SystemMessage(content="You are a math tutor. Be concise.")
]

# Simulate multiple questions
questions = [
    "What is 15 + 27?",
    "What is 8 * 9?",
    "What is 100 / 4?",
    "What is 12 squared?",
    "What is the square root of 144?",
    "What is 50 - 18?",
]

for i, question in enumerate(questions, 1):
    messages.append(HumanMessage(content=question))
    response = llm.invoke(messages)
    messages.append(AIMessage(content=response.content))
    
    print(f"Q{i}: {question}")
    print(f"A{i}: {response.content}")
    
    # Trim if exceeding max
    if len(messages) > MAX_MESSAGES:
        # Keep system message + most recent messages
        messages = [messages[0]] + messages[-(MAX_MESSAGES-1):]
        print(f"  [Trimmed to {len(messages)} messages]")
    
    print()

## 5. Combining Patterns

Use buffer memory with a sliding window for the best of both worlds.

In [ ]:
class SlidingWindowMemory:
    """Memory that keeps only the N most recent exchanges."""
    
    def __init__(self, max_exchanges=5):
        self.max_exchanges = max_exchanges
        self.messages = []
    
    def add_exchange(self, human_msg: str, ai_msg: str):
        self.messages.append(HumanMessage(content=human_msg))
        self.messages.append(AIMessage(content=ai_msg))
        
        # Keep only last N exchanges (2 messages per exchange)
        max_msgs = self.max_exchanges * 2
        if len(self.messages) > max_msgs:
            self.messages = self.messages[-max_msgs:]
    
    def get_messages(self):
        return self.messages
    
    def clear(self):
        self.messages = []

# Test it
sliding_memory = SlidingWindowMemory(max_exchanges=3)

for i in range(5):
    sliding_memory.add_exchange(
        f"This is question number {i+1}",
        f"This is answer number {i+1}"
    )

print(f"After 5 exchanges, memory contains {len(sliding_memory.get_messages())} messages:")
for msg in sliding_memory.get_messages():
    print(f"  {msg.type}: {msg.content}")

## Summary

**Memory Pattern Comparison:**

| Pattern | Best For | Pros | Cons |
|---------|----------|------|------|
| Manual Message History | Simple use cases | Full control, straightforward | Manual management |
| ConversationBufferMemory | Short conversations | Easy to use, complete history | Memory grows unbounded |
| ConversationSummaryMemory | Long conversations | Bounded memory size | Loses details, extra LLM calls |
| Context Window Management | Token-conscious apps | Predictable memory usage | Recent messages only |
| Sliding Window | Production chatbots | Balance of history and efficiency | May lose important context |

**When to use each:**
- **Manual History**: Prototyping, debugging
- **Buffer Memory**: Customer support (few turns)
- **Summary Memory**: Research assistants, long sessions
- **Sliding Window**: High-volume chatbots
- **Combination**: Complex applications needing both detail and efficiency